In [1]:
import pandas as pd
import numpy as np

In [2]:
data = {
    'Key 1': ['ABC123', 'ABC123', 'ABC456', 'ABC789', '', '', ''],
    'Key 2': ['', '', 'ABC456', 'ABC789', 'PQR123', 'PQR123', ''],
    'Key 3': ['', '', '', '', '', '', 'XYZ123'],
    'Col 1': ['Fixed Leg', 'Floating Leg', 'Some value', 'Some value', 'Some value', 'Some value', 'Some value'],
    'Col 2': ['Some value 1', 'Some value 2', 'Some value', 'Some value', 'Some value', 'Some value', 'Some value'],
    'Col 3': ['Some value', 'Some value', 'Some value', 'Some value', 'Fixed Leg', 'Floating Leg', 'Some value'],
    'Col 4': ['Some value', 'Some value', 'Some value', 'Some value', 'Some value', 'Some value', 'Some value'],
    'Col 5': ['Some value', 'Some value', 'Some value', 'Some value', 'Some value', 'Some value', 'Some value']
}

df = pd.DataFrame(data)

In [3]:
df

,Key 1,Key 2,Key 3,Col 1,Col 2,Col 3,Col 4,Col 5
0,ABC123,,,Fixed Leg,Some value 1,Some value,Some value,Some value
1,ABC123,,,Floating Leg,Some value 2,Some value,Some value,Some value
2,ABC456,ABC456,,Some value,Some value,Some value,Some value,Some value
3,ABC789,ABC789,,Some value,Some value,Some value,Some value,Some value
4,,PQR123,,Some value,Some value,Fixed Leg,Some value,Some value
5,,PQR123,,Some value,Some value,Floating Leg,Some value,Some value
6,,,XYZ123,Some value,Some value,Some value,Some value,Some value


In [14]:
def flatten_dataframe(input_df, key_columns):
   # Melt the dataframe to long format while preserving key columns
    df_melt = input_df.melt(id_vars=key_columns, 
                            var_name='variable', 
                            value_name='value')
    
    print(df_melt)
    
    # Create a helper column for cumcount.
    # This helps track repeated occurrences of a value in the melted dataframe.
    df_melt['helper'] = df_melt.groupby(key_columns + ['variable']).cumcount() + 1
    
    print('--------------------------------------------')
    
    print(df_melt)

    # Pivot the dataframe back into wide format using the keys and the helper column.
    # This creates a wide format dataframe where each unique value in a key has its own column.
    df_pivot = df_melt.pivot_table(index=key_columns, 
                                   columns=['variable', 'helper'], 
                                   values='value', 
                                   aggfunc='first')

    # Flatten the MultiIndex columns. 
    # Here we join the two levels of columns (variable and helper) into one level.
    df_pivot.columns = [f'{i}_{j}' if j != 1 else f'{i}' for i, j in df_pivot.columns]

    # Reset the index so that our key columns are no longer the index of our dataframe
    df_pivot = df_pivot.reset_index()

    return df_pivot

In [15]:
keys = ['Key 1', 'Key 2', 'Key 3']
flatten_dataframe(df, keys)

     Key 1   Key 2   Key 3 variable         value
0   ABC123                    Col 1     Fixed Leg
1   ABC123                    Col 1  Floating Leg
2   ABC456  ABC456            Col 1    Some value
3   ABC789  ABC789            Col 1    Some value
4           PQR123            Col 1    Some value
5           PQR123            Col 1    Some value
6                   XYZ123    Col 1    Some value
7   ABC123                    Col 2  Some value 1
8   ABC123                    Col 2  Some value 2
9   ABC456  ABC456            Col 2    Some value
10  ABC789  ABC789            Col 2    Some value
11          PQR123            Col 2    Some value
12          PQR123            Col 2    Some value
13                  XYZ123    Col 2    Some value
14  ABC123                    Col 3    Some value
15  ABC123                    Col 3    Some value
16  ABC456  ABC456            Col 3    Some value
17  ABC789  ABC789            Col 3    Some value
18          PQR123            Col 3     Fixed Leg


,Key 1,Key 2,Key 3,Col 1,Col 1_2,Col 2,Col 2_2,Col 3,Col 3_2,Col 4,Col 4_2,Col 5,Col 5_2
0,,,XYZ123,Some value,NaN,Some value,NaN,Some value,NaN,Some value,NaN,Some value,NaN
1,,PQR123,,Some value,Some value,Some value,Some value,Fixed Leg,Floating Leg,Some value,Some value,Some value,Some value
2,ABC123,,,Fixed Leg,Floating Leg,Some value 1,Some value 2,Some value,Some value,Some value,Some value,Some value,Some value
3,ABC456,ABC456,,Some value,NaN,Some value,NaN,Some value,NaN,Some value,NaN,Some value,NaN
4,ABC789,ABC789,,Some value,NaN,Some value,NaN,Some value,NaN,Some value,NaN,Some value,NaN
